In [1]:



import logging
import numpy as np

import matplotlib.pyplot as plt
from importlib import reload
import random

import tensorflow as tf
import importlib
import MCAgent as agent
import Model as Con4
import All_In_One_environment as env
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, BatchNormalization, Activation, LeakyReLU, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

2021-12-08 12:04:20.749507: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [8]:
#local imports reload
importlib.reload(agent)
importlib.reload(Con4)
importlib.reload(env)

<module 'All_In_One_environment' from '/home/jevans40/AI/AIClass2021/Project/Group2-s_UW_Intro_AI_Project/Project_attempt_one/Project_attempt_one/All_In_One_environment.py'>

In [3]:
#settings:
#Settings
inputDim = (6,7,2)
outputDim = (7)
filterSize = 20
kernelDim = (4,4)
convLayers = 4
regConst = 0.0001
learningRate = 0.001
iteration = 1

Memory = []

In [4]:
def RunTwoAgents(Agent1,Agent2, Games, Verbosity = 0):
    memories = []
    for game in range(Games):
        print("Playing game {} out of {}".format(game,Games))
        winner = PlayGame(Agent1,Agent2,Verbosity)
        memories += Agent1.GetMemory()
        memories += Agent2.GetMemory()
        Agent1.ClearMemory()
        Agent2.ClearMemory()   

        tmp = Agent2
        Agent2 = Agent1
        Agent1 = tmp
    return memories
    #Return memory to train on

#Verbosity controls how much information to output
#0: None
#1: Winner/Loser
#2: Memory Output
def PlayGame(Agent1,Agent2,Verbosity=0):
    Agent1.SetPlayer(-1)
    Agent2.SetPlayer(1)
    done = False
    memories = []
    newGame = env.ConnectFour(1)
    num = random.randint(0,6)
    newGame.step(num,1)
    while not done:
        action = Agent1.GetAction(newGame)
        #Add some level of randomness so that we can get more varied cases
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,Agent1.player)
        memories.append(action)
        if Verbosity > 1:
            print("Player -1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player -1 has won the game!")
            return -1

        action = Agent2.GetAction(newGame)
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,Agent2.player)
        if Verbosity > 1:
            print("Player 1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player 1 has won the game!")
            return 1
        if Verbosity > 2:
            print(np.array(newGame.get_current_state()).reshape(6,7))
    
    #Return game result

def TrainModel(Memories, toFitAgent):
    for i in range(5):
        minbatch = random.sample(Memories,min(250,len(Memories)))

        trainingStates = np.array([Con4.ReshapeToModel(item[0],item[3]) for item in Memories])
        trainingTargets = np.array([item[1] for item in Memories])

        fit = toFitAgent.model.fit(trainingStates,trainingTargets,epochs=5,verbose=1,validation_split=0,batch_size = 32)

        toFitAgent.model.save("ModelCheckPointStart")
    return round(fit.history["loss"][0],4)

In [5]:
testModel = Con4.CN4Model(inputDim,outputDim,filterSize,kernelDim,regConst,convLayers,learningRate)
testModel.InitModel()


2021-12-08 12:04:32.921412: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-12-08 12:04:33.831274: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:968] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1f:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-08 12:04:33.831446: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:1f:00.0 name: NVIDIA GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.845GHz coreCount: 46 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.23GiB/s
2021-12-08 12:04:33.831481: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-08 12:04:33.903303: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-12-08 12:04:33.944287: I tensorflow/stream_executo

In [9]:
Agent1 = agent.ReinforcementAgent(testModel,-1)
Agent2 = agent.ReinforcementAgent(testModel,1)

In [ ]:
Mem = RunTwoAgents(Agent1, Agent2,5, Verbosity=2)
print(Mem)

In [ ]:
TrainModel(Mem,testModel)

In [10]:
while True:
    Mem = RunTwoAgents(Agent1, Agent2,10, Verbosity=0)
    TrainModel(Mem,testModel)

Playing game 0 out of 7
truth
[0.19128847 0.14263221 0.12596594 0.11970318 0.19128847 0.11452521
 0.11452521]
prediction
[0.06823576 0.00589579 0.29920635 0.29525608 0.08007082 0.06724966
 0.18408552]
truth
[0.14750434 0.1411306  0.15003353 0.14245269 0.15084323 0.14750434
 0.12048095]
prediction
[0.0472114  0.04012929 0.23374116 0.2748812  0.08967698 0.21956275
 0.0947973 ]
